# Introduction
In 1953, Metropolis, Rosenbluth, and the Teller couple published a paper titled *Equation of State Calculations by Fast Computing Machines* proposing a random algorithm for using computers to solve the statistical properties of strongly correlated many-body systems. In this article, they performed calculations on a two-dimensional rigid sphere model. The key idea of the paper is to view the phase space of the canonical ensemble as a probability space. If the coordinates of N particles are given, one can calculate the energy (potential energy) of this configuration. Since the potential energy depends only on the positions and not on the velocities, the integration over momentum degrees of freedom in the phase space can be eliminated. **Hence, the statistical properties of the system are only related to the potential energy part**.
$$
<A> = \frac{\int A e^{-E/ \tau} d^{3N}q}{Z}
$$
where $Z$ is the partition function, $E$ is the total energy, and $\tau$ is the temperature. The key idea of the algorithm is to use a random walk to sample the phase space.

An intuitive approach is to randomly assign particle coordinates, calculate the physical quantity $A$ and the energy $E$ of the configuration, and then apply the statistical weight $e^{-E/\tau}$. Finally, the weighted average of the physical quantity $A$ is obtained. This method is feasible for sparse systems, but for dense systems, the majority of configurations have extremely small weights (due to high energy caused by overlap), making it difficult to sample configurations with large weights. As a result, the algorithm's efficiency becomes extremely low. The authors proposed a modified method known as the famous **Metropolis algorithm**: instead of randomly and uniformly sampling configurations and assigning them weights $e^{-E/τ}$, configurations are selected according to their probabilities, which are proportional to  $e^{-E/τ}$. This way, it is more efficient to sample configurations with larger weights and improve the algorithm's efficiency.Specifically, the algorithm is as follows:
- 1 Randomly initialize the coordinates of N particles and calculate their energy.
- 2 Randomly select a particle and move it from qi to qi+αξ, where α is the step size (manually set) and ξ is a uniformly distributed random number in the range [-1, 1]. To reduce boundary effects, the authors used periodic boundary conditions.
- 3 Calculate the energy difference (ΔE) between the new configuration and the original configuration. If the energy of the new configuration is lower, it is "accepted." If the energy is higher, generate a random number ε from a uniform distribution in the range [0, 1]. If ε is less than e^(-ΔE/τ), the new configuration is "accepted"; otherwise, it is "rejected" and the system is restored to the configuration before the move.
- 4 Regardless of whether the new configuration is accepted or rejected, the configuration after steps 2 and 3 is considered a sampled configuration.
- Repeat steps 2-4 multiple times until a sufficient number of sampled configurations (denoted as M) are obtained to control statistical errors within an acceptable range.

For the desired physical quantity, compute the unweighted sample average, which represents the result of a single "experiment". Sometimes, higher-order moments of the physical quantity may also be needed, and these moments are likewise results of a single "experiment".
The above describes the Metropolis algorithm, with step 4 being its core. It is evident that this is a Markov process, where each transition (steps 2-4) yields a state, and the transition probability P(Xn+1=x|Xn=xn) from the nth state to the n+1th state is independent of n.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.patches as pch

In [2]:
x=np.zeros((27,10000))
y=np.zeros((27,10000))
z=np.zeros((27,10000))
vx=np.zeros((27,10000))
vy=np.zeros((27,10000))
vz=np.zeros((27,10000))
ax=np.zeros((27,10000))
ay=np.zeros((27,10000))
az=np.zeros((27,10000))

In [3]:
location=np.array([[2.5,2.5,2.5],
                 [1.5,2.5,2.5],
                 [3.5,2.5,2.5],
                 [2.5,1.5,2.5],
                 [2.5,3.5,2.5],
                 [2.5,2.5,1.5],
                 [2.5,2.5,3.5],
                 [3.5,3.5,2.5],
                 [3.5,1.5,2.5],
                 [1.5,3.5,2.5],
                 [1.5,1.5,2.5],
                 [3.5,2.5,3.5],
                 [3.5,2.5,1.5],
                 [1.5,2.5,3.5],
                 [1.5,2.5,1.5],
                 [2.5,3.5,3.5],
                 [2.5,3.5,1.5],
                 [2.5,1.5,3.5],
                 [2.5,1.5,1.5],
                 [3.5,3.5,3.5],
                 [3.5,3.5,1.5],
                 [3.5,1.5,3.5],
                 [1.5,3.5,3.5],
                 [1.5,1.5,3.5],
                 [1.5,3.5,1.5],
                 [3.5,1.5,1.5],
                 [1.5,1.5,1.5],])

In [4]:
for i in range(27):
    x[i][0]=location[i][0]
    y[i][0]=location[i][1]
    z[i][0]=location[i][2]

In [5]:
def compare(xi,xj):#计算最近镜像
    if abs(xi-xj)<=abs(xi-xj+L):
        if abs(xi-xj)<=abs(xi-xj-L):
            px=0
            dx=abs(xi-xj)
        else:
            px=+1
            dx=abs(xi-xj-L)
    else:
        if abs(xi-xj+L)<=abs(xi-xj-L):
            px=-1
            dx=abs(xi-xj+L)
        else:
            px=+1
            dx=abs(xi-xj-L)
    return px,dx

In [6]:
def cal_r(xi,yi,zi,xj,yj,zj):
    px,dx=compare(xi,xj)
    py,dy=compare(yi,yj)
    pz,dz=compare(zi,zj)
    r=math.sqrt(dx*dx+dy*dy+dz*dz)
    return r,px,py,pz#px,py,pz表示镜像的位置    

In [76]:
import random
L=5
rc=2.5
temperature=5

In [10]:
up=np.zeros(10000)

In [11]:
for i in range(1):
    for j in range(27):
        for k in range(27):
            if j==k:
                continue
            rjk,px,py,pz=cal_r(x[j][i],y[j][i],z[j][i],x[k][i],y[k][i],z[k][i])
            up[i]+=2*(math.pow(rjk,-12)-math.pow(rjk,-6))

In [21]:
temp1=(random.random()-0.5)*L*2
temp2=(random.random()-0.5)*L*2
temp3=(random.random()-0.5)*L*2
temp1,temp2,temp3

(2.5082670033083474, 0.012669371113939176, -1.1880386486439343)

In [77]:
math.exp(-2)

0.1353352832366127

In [78]:
for i in range(9999):
    #print(i)
    temp1=(random.random()-0.5)*L*2
    temp2=(random.random()-0.5)*L*2
    temp3=(random.random()-0.5)*L*2
    jt=random.randint(0,26)#被选中的粒子序号
    for j in range(27):
        x[j][i+1]=x[j][i]
        y[j][i+1]=y[j][i]
        z[j][i+1]=z[j][i]
    x[jt][i+1]=(x[jt][i]+temp1)%L
    y[jt][i+1]=(y[jt][i]+temp2)%L
    z[jt][i+1]=(z[jt][i]+temp3)%L
    
    for j in range(27):
        for k in range(27):
            if j==k:
                continue
            rjk,px,py,pz=cal_r(x[j][i+1],y[j][i+1],z[j][i+1],x[k][i+1],y[k][i+1],z[k][i+1])
            up[i+1]+=2*(math.pow(rjk,-12)-math.pow(rjk,-6))
    if(up[i+1]>up[i]):
        delta_E=up[i+1]-up[i]
        q=random.random()
        if(q>math.exp(-delta_E/temperature)):
            x[jt][i+1]=x[jt][i]
            y[jt][i+1]=y[jt][i]
            z[jt][i+1]=z[jt][i]
            up[i+1]=up[i]
    

In [86]:
p=np.zeros(10000)
for i in range(10000):
    for j in range(27):
        for k in range(27):
            if j==k:
                continue
            rjk,px,py,pz=cal_r(x[j][i],y[j][i],z[j][i],x[k][i],y[k][i],z[k][i])
            p[i]+=2*(-12*math.pow(rjk,-12)+6*math.pow(rjk,-6))#在计算时已经乘了1/2，因为每两个粒子之间的作用势被计算两次

## Compute the ensemble average over the first 10,000 samples.

In [87]:
N=27
V=L*L*L
p_ex=0
for i in range(10000):
    p_ex+=p[i]
p_ex=p_ex/(30000*V)
p_final=N*temperature/V-p_ex

In [88]:
p_final

1.3364947249313484

## Compute the ensemble average over the last 5,000 samples to ensure convergence of the Markov process to the stationary distribution.

In [89]:
N=27
V=L*L*L
p_ex=0
for i in range(5000,10000):
    p_ex+=p[i]
p_ex=p_ex/(15000*V)
p_final=N*temperature/V-p_ex

In [90]:
p_final

1.2974667972170972

In [91]:
N*temperature/V

1.08

# Chemical Potential 
Through the previous two ipynb files, we have learned that as long as we can express a statistical quantity we want to study as an ensemble average, we can compute it using molecular dynamics simulations or Monte Carlo methods. However, we cannot express the free energy and partition function of a system as ensemble averages, so we cannot directly obtain these quantities through simulation methods. Nevertheless, we can still find a way to express the difference between these quantities and the statistical quantities of an ideal gas as ensemble averages. Below, we will introduce the **Widom method** (also known as the **Particle Insertion method**) for calculating the chemical potential in the canonical ensemble.

### Particle insertion method

In the Widom method, we assume that our system already contains N-1 particles, and we want to calculate the chemical potential of inserting an additional particle into this system. We can express this chemical potential as a difference, namely the difference between the chemical potential of a system with N particles and the chemical potential of a system with N-1 particles.

Using the particle insertion method, we can insert a new particle into the system and calculate its energy and interactions with the other particles. Then, by averaging over all possible insertion positions, we can convert these energy differences into ensemble averages. Ultimately, we can calculate the chemical potential in the canonical ensemble using the average energy differences obtained from the simulation.

The particle insertion method is a commonly used technique for calculating the chemical potential in the canonical ensemble. It allows us to obtain the system's chemical potential through simulation, even though we cannot directly express it as an ensemble average. This method has broad applications in theoretical and computational chemistry.

When N is large, we can make an important approximation:
$$
F = -\tau log Z_N, \mu = \left(\frac{\partial F}{\partial N}\right )_{V,T} \approx -\tau log \frac{Z_{N+1}}{Z_N}
$$
The expression for the partition function is: 
$$
Z(N,V,\tau) = \frac{V^{N}}{\Lambda^{3N}N!} \int e^{-\beta E(\boldsymbol{s}^{N};L)} d^{3N}s, \quad dr_{(i)} = Lds_{i}
$$
Substituting the approximate expression for the chemical potential μ, we have:
$$
\mu = -\tau log\left(\frac{V/\Lambda^{3}}{N+1}\right ) -\tau log \frac{\int e^{-\beta E(\boldsymbol{s}^{N+1};L)} d^{N+1}\boldsymbol{s}}{\int e^{-\beta E(\boldsymbol{s}^{N};L)} d^{N}\boldsymbol{s}} \equiv \mu_{ideal}+\mu_{ex} 
$$
The part concerning the ideal gas can be directly calculated. Now let's calculate the remaining part. Note that $μ_{ex}$ can be expressed as an ensemble average:
$$
Let \quad \Delta E = E(\boldsymbol{s}^(N+1)) - E(\boldsymbol{s}^N)
$$
$$
μ_{ex} = -\tau log\left(\int d\boldsymbol{s}^{N+1} <exp(-\beta \Delta E)>_{N}\right)
$$
Therefore, in a Monte Carlo simulation of an N-particle system, we only need to randomly insert a virtual particle (with its position uniformly distributed within the box) at a certain frequency. Calculate the additional potential energy $\Delta E$ caused by this particle and then take an average of the corresponding Boltzmann factor $exp(-\beta \Delta E)$. Multiply the result by $-\tau$ after taking the logarithm to obtain $μ_{ex}$, and subsequently obtain the chemical potential of the system.


In [92]:
up_extra=np.zeros(10000)
for i in range(5000,10000):
    temp1=random.random()*L
    temp2=random.random()*L
    temp3=random.random()*L
    for j in range(27):
        rjk,px,py,pz=cal_r(temp1,temp2,temp3,x[j][i],y[j][i],z[j][i])
        up_extra[i]+=4*(math.pow(rjk,-12)-math.pow(rjk,-6))
        

In [93]:
upx=0
for i in range(5000,10000):
    upx+=math.exp(-up_extra[i]/temperature)
upx=upx/5000

In [94]:
chemistry_potential=-temperature*math.log(upx)

In [95]:
chemistry_potential

1.8683857758632152